In [1]:
import os
sys.path.append(os.path.abspath(".."))
from shopping_agent import run_shopping_agent

In [2]:
# instruction = "لطفاً دراور چهار کشو (کد D14) را برای من تهیه کنید."
# instruction = "عرض پارچه تریکو جودون 1/30 لاکرا گردباف نوریس به رنگ زرد طلایی چقدر است؟"
# instruction = "کمترین قیمت در این پایه برای گیاه طبیعی بلک گلد بنسای نارگل کد ۰۱۰۸ چقدر است؟"
instruction = "What is the most popular product?"
result = await run_shopping_agent(instruction)

Step 1: Understand the task:
The user wants to know the most popular product. Popular product can be interpreted in different ways such as most viewed product, most clicked product, or most searched product. The most reasonable approach is to consider popularity measured by user interactions (views and clicks) on base products.

Step 2: Determine output parts:
We need to provide a message describing the most popular product, which could include its Persian name and the measure of popularity (like views or clicks count). base_random_keys and member_random_keys will be null since user asked general info.

Step 3: Breakdown subtasks:
- Decide the popularity metric: number of final clicks on base products is the best indicator.
- Count clicks per base product by joining clicks with base_views (to get base product from base_view).
- Find the base product with the highest clicks.
- Retrieve product details (e.g. Persian name) from base_products table.

Step 4: Tools and queries:
- Use execut

In [3]:
result

AgentRunResult(output=ShoppingResponse(message='The most popular product based on user clicks is "سرخ کن فیلیپس مدل HD9270 ظرفیت ۶.۲ لیتر بدون روغن" with 24 clicks. This indicates it is highly favored by users on Torob.', base_random_keys=None, member_random_keys=None))

In [4]:
dict(result.output)

{'message': 'The most popular product based on user clicks is "سرخ کن فیلیپس مدل HD9270 ظرفیت ۶.۲ لیتر بدون روغن" with 24 clicks. This indicates it is highly favored by users on Torob.',
 'base_random_keys': None,
 'member_random_keys': None}

In [30]:
result.all_messages()

[ModelRequest(parts=[SystemPromptPart(content="\n\nYou are an AI Shopping Assistant for Torob. Your job is to answer user instructions by retrieving product information.\n\n\n\nYou have access to the following tools:\n\n1. execute_sql(query: str) -> list[RealDictRow]: Executes a PostgreSQL query and returns results.\n2. generate_sql_query(instruction: str) -> tuple[str, str]: Generates a PostgreSQL query from natural language.\n3. build_like_query(table_name: str, column_name: str, variable_query: str, limit: int = 10) -> str:\n   Returns a SQL query string that searches for rows in a table where a column contains a given substring (case-insensitive).\n   I would suggest to let limit be at least 3.\n\n\n\n  0. Avoid redundant calls to generate_sql_query if it's actually unncessary.\n  1. Use the already given queries if it's sufficient, and avoid calling generate_sql_query to create a new query.\n  2. Execute SQL if necessary.\n  3. Fill the Pydantic response for your final answer.\n\n